In [29]:
# The aim of this code is to generate 
# 1. a set of outcomes used in the analysis of hukou reform determinants
# 2. a set of covariates used in the analysis of hukou reform impacts - provincial level
# 3. a set of covariates used in the analysis of hukou reform impacts - city level

# The data set used in this code is generated by the author based on original policy documents issued by each Chinese province

import pandas as pd
import numpy as np


# 0. Read the data and assign weight to each row

policy = pd.read_excel("PolicyData.xlsx", index_col=0)
policy['score'] = policy['res_all'] * policy['mag_all']
policy = policy[policy['score'] < 5]

mig_weight = {10:1, 20:0.73, 21:0.63, 22:0.63, 23:0.11, 30:0.56, 31:0.45, 32:0.11, 33:0.11, 40:0.01, 41:0.01, 42:0.01, 
              50:1.00, 51:0.73, 52:0.17, 53:0.1, 54:0.17, 61:0.11, 62:0.63, 63:0.01, 64:0.01, 65:0.01}

policy['weight'] = [mig_weight[i] for i in policy['mig_group']]
policy['wscore'] = policy['score'] * policy['weight']


def subset(data, col, v1, v2 = None, v3 = None, v4 = None, v5 = None, v6 = None, v7 = None):
    return data.loc[(data[col] == v1) | (data[col] == v2) | (data[col] == v3) | (data[col] == v4) | 
                    (data[col] == v5) | (data[col] == v6) | (data[col] == v7), :]

def ts(subset = policy, score_type = "score", code_value = 1, by_col = "pol_area"):
    return subset.loc[subset[by_col] == code_value, score_type].groupby(level=0).sum()


def avg(subset = policy, score_type = "score", code_value = 1, by_col = "pol_area"):
    return subset.loc[subset[by_col] == code_value, score_type].groupby(level=0).mean()


def count(subset = policy, code_value = 1, by_col = "pol_area", score_type = "res_all"):
    return subset.loc[subset[by_col] == code_value, score_type].groupby(level=0).count()


# 1. Outcomes used for determinants analysis

# The set of outcomes refers to the policy scores disaggregated by policy areas (selection & integration), including

# A. total and average selection scores (weighted and unweighted)
# B. total and average selection scores targeting low/high-skilled migrants (weighted and unweighted)
# C. total and average selection scores by different city sizes (weighted and unweighted)
# D. total and average integration scores (unweighted)
# E. Number of more lenient selection and integration measures

# 1.1 Construct A, D & E

ts_sel = ts()
as_sel = avg()

wts_sel = ts(policy, "wscore")
was_sel = avg(policy, "wscore")

ts_int = ts(policy, "score", 2)
as_int = avg(policy, "score", 2)

num_ls = count(subset(policy, "res_all", -1))
num_li = count(subset(policy, "res_all", -1), 2)

                
# 1.2 Construct B

lts_sel = ts(subset(policy, "mig_group", 20, 21, 22, 30, 31))
las_sel = avg(subset(policy, "mig_group", 20, 21, 22, 30, 31))
               
wlts_sel = ts(subset(policy, "mig_group", 20, 21, 22, 30, 31), "wscore")
wlas_sel = avg(subset(policy, "mig_group", 20, 21, 22, 30, 31), "wscore")

hts_sel = ts(subset(policy, "mig_group", 23, 32, 33))
has_sel = avg(subset(policy, "mig_group", 23, 32, 33))
               
whts_sel = ts(subset(policy, "mig_group", 23, 32, 33), "wscore")
whas_sel = avg(subset(policy, "mig_group", 23, 32, 33), "wscore")               
            
               
# 1.3 Construct C

ts_small = ts(subset(policy, "des", 11))
as_small = avg(subset(policy, "des", 11))

wts_small = ts(subset(policy, "des", 11), "wscore")
was_small = avg(subset(policy, "des", 11), "wscore")


ts_medium = ts(subset(policy, "des", 12))
as_medium = avg(subset(policy, "des", 12))

wts_medium = ts(subset(policy, "des", 12), "wscore")
was_medium = avg(subset(policy, "des", 12), "wscore")


ts_big = ts(subset(policy, "des", 13, 14, 15))
as_big = avg(subset(policy, "des", 13, 14, 15))

wts_big = ts(subset(policy, "des", 13, 14, 15), "wscore")
was_big = avg(subset(policy, "des", 13, 14, 15), "wscore")


ts_mega = ts(subset(policy, "des", 16))
as_mega = avg(subset(policy, "des", 16))

wts_mega = ts(subset(policy, "des", 16), "wscore")
was_mega = avg(subset(policy, "des", 16), "wscore")


ts_big_mega = ts(subset(policy, "des", 13, 14, 15, 16))
as_big_mega = avg(subset(policy, "des", 13, 14, 15, 16))

wts_big_mega = ts(subset(policy, "des", 13, 14, 15, 16), "wscore")
was_big_mega = avg(subset(policy, "des", 13, 14, 15, 16), "wscore")


df1 = pd.concat([ts_sel, as_sel, wts_sel, was_sel, lts_sel, las_sel, wlts_sel, wlas_sel,
                ts_small, ts_medium, ts_big, ts_mega, ts_big_mega, as_small, as_medium, as_big, as_mega, as_big_mega, 
                wts_small, wts_medium, wts_big, wts_mega, wts_big_mega, was_small, was_medium, was_big, was_mega, was_big_mega,
                ts_int, as_int, num_ls, num_li], axis=1).fillna(0)

df1.columns = ["ts_sel", "as_sel", "wts_sel", "was_sel", "lts_sel", "las_sel", "wlts_sel", "wlas_sel",
              "ts_small", "ts_medium", "ts_big", "ts_mega", "ts_big_mega", 
              "as_small", "as_medium", "as_big", "as_mega", "as_big_mega",
              "wts_small", "wts_medium", "wts_big", "wts_mega", "wts_big_mega",
              "was_small", "was_medium", "was_big", "was_mega", "was_big_mega",
              "ts_int", "as_int", "num_ls", "num_li"]

df1.to_excel(r"C:\Users\User\Desktop\Clustering\Scores_Province_Area.xlsx")



# 2. Covariates used for Impact analysis - provincial level

# The set of covariates refers to the (unweighted) total and average policy scores disaggregated by the following tools:

# A. access to local hukou (access)
# B. points system (ps)
# C. residential permit (rp)
# D. urban-rural unification (unify)
# E. public service-related tools (ser)
# F. insurance-related  tools (insr)


ts_lower = ts(policy, "score", 11, "pol_tool")
as_lower = avg(policy, "score", 11, "pol_tool")

ts_ps = ts(policy, "score", 12, "pol_tool")
as_ps = avg(policy, "score", 12, "pol_tool")

ts_rp = ts(policy, "score", 21, "pol_tool")
as_rp = avg(policy, "score", 21, "pol_tool")

ts_unify = ts(policy, "score", 23, "pol_tool")
as_unify = avg(policy, "score", 23, "pol_tool")
 
ts_ser = ts(subset(policy, "pol_tool", 30, 31, 36, 37, 38, 39, 54), "score", 2)
as_ser = avg(subset(policy, "pol_tool", 30, 31, 36, 37, 38, 39, 54), "score", 2)

ts_insr = ts(subset(policy, "pol_tool", 32, 33, 34, 35), "score", 2)
as_insr = avg(subset(policy, "pol_tool", 32, 33, 34, 35), "score", 2)

ts_edu = ts(policy, "score", 31, "pol_tool")
as_edu = avg(policy, "score", 31, "pol_tool")

df2 = pd.concat([ts_lower, as_lower, ts_ps, as_ps, 
                 ts_rp, as_rp, ts_unify, as_unify, ts_ser, as_ser, ts_insr, as_insr, ts_edu, as_edu], axis=1).fillna(0)

df2.columns = ["ts_lower", "as_lower", "ts_ps", "as_ps",
               "ts_rp", "as_rp", "ts_unify", "as_unify", "ts_ser", "as_ser", "ts_insr", "as_insr", "ts_edu", "as_edu"]

df2.to_excel(r"C:\Users\User\Desktop\Clustering\Scores_Province_Tools.xlsx")



# 3. Covariates used for Impact analysis - city level

# The set of covariates consists primarily of the previously constructed variables disaggregated by the following city sizes:

# A. small city (population < 500,000)
# B. medium (500,000 ~ 1 million)
# C. big I (1 ~ 3 million)
# D. big II (3 ~ 5 milion)
# E. mega ( > 5 million)

des = {11:"small", 12:"medium", 13:"big I", 14:"big II", 15:"big", 16:"mega", 
       17:"small & medium", 18:"medium & big", 19:"big & mega", 99:"all sizes"}

small = subset(policy, "des", 11, 17, 99)
medium = subset(policy, "des", 12, 17, 18, 99)
big1 = subset(policy, "des", 13, 15, 18, 19, 99)
big2 = subset(policy, "des", 14, 15, 18, 19, 99)
mega = subset(policy, "des", 16, 19, 99)


def gen_df(df, pre):
    return pd.DataFrame({pre + "ts_sel": ts(df),
                         pre + "as_sel": avg(df),
                         pre + "ts_access": ts(df, "score", 11, "pol_tool"),
                         pre + "as_access": avg(df, "score", 11, "pol_tool"),
                         pre + "ts_ps": ts(df, "score", 12, "pol_tool"),
                         pre + "as_ps": avg(df, "score", 12, "pol_tool"),
                         pre + "wts_sel": ts(df, "wscore"), 
                         pre + "was_sel": avg(df, "wscore"),
                         pre + "lts_sel": ts(subset(df, "mig_group", 20, 21, 22, 30, 31)),
                         pre + "las_sel": avg(subset(df, "mig_group", 20, 21, 22, 30, 31)),
                         pre + "hts_sel": ts(subset(df, "mig_group", 23, 32, 33)),
                         pre + "has_sel": avg(subset(df, "mig_group", 23, 32, 33)),
                         pre + "num_ls": count(subset(policy, "res_all", -1)),
                         pre + "num_li": count(subset(policy, "res_all", -1), 2),
                         pre + "ts_int": ts(df, "score", 2),
                         pre + "as_int": avg(df, "score", 2),
                         pre + "ts_rp": ts(df, "score", 21, "pol_tool"),
                         pre + "as_rp": avg(df, "score", 21, "pol_tool"),
                         pre + "ts_unify": ts(df, "score", 23, "pol_tool"),
                         pre + "as_unify": avg(df, "score", 23, "pol_tool"),
                         pre + "ts_ser": ts(subset(df, "pol_tool", 30, 31, 36, 37, 38, 39, 54), "score", 2),
                         pre + "as_ser": avg(subset(df, "pol_tool", 30, 31, 36, 37, 38, 39, 54), "score", 2),
                         pre + "ts_insr": ts(subset(df, "pol_tool", 32, 33, 34, 35), "score", 2),
                         pre + "as_insr": avg(subset(df, "pol_tool", 32, 33, 34, 35), "score", 2)}).fillna(0)


df3 = pd.concat([gen_df(small, "small_"), gen_df(medium, "medium_"),
                 gen_df(big1, "big1_"),  gen_df(big2, "big2_"), gen_df(mega, "mega_")], axis = 1)

df3.to_excel(r"C:\Users\User\Desktop\Clustering\Scores_City.xlsx")